# Make solar forcing

As we have volcanic monthly, we also want solar monthly. Follow the AR6 method, and set solar forcing to be zero over the 1750-2014 mean.

In [ ]:
import os

import matplotlib.pyplot as pl
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import pooch
from scipy.interpolate import interp1d

In [ ]:
satire14c = pooch.retrieve(
    url = "https://sharebox.lsce.ipsl.fr/index.php/s/LpiCUCkSmx0P6bb/download?path=%2F&files=SSI_14C_cycle_yearly_cmip_v20160613_fc.nc",
    known_hash="7a973502cdcfa492f2006bd1283827f8b0ef632efd7cf32a2d67ec350bfa2da1"
)

In [ ]:
nc = Dataset(satire14c)
wl_bin = nc.variables['wavelength_bin'][:]
time   = nc.variables['time'][:]
ssi    = nc.variables['ssi'][:]
nc.close()

In [ ]:
time[8504], time[8604], time[8605]

In [ ]:
time_17495_18500 = time[8504:8606]

In [ ]:
tsi_yearly_17495_18500 = np.zeros(102)
for idx in range(8504, 8606):
    tsi_yearly_17495_18500[idx-8504] = np.sum(ssi[idx,:]*wl_bin)

In [ ]:
cmip6 = pooch.retrieve(
    url = "http://aims3.llnl.gov/thredds/fileServer/user_pub_work/input4MIPs/CMIP6/CMIP/SOLARIS-HEPPA/SOLARIS-HEPPA-3-2/atmos/mon/multiple/gm/v20170103/solarforcing-ref-mon_input4MIPs_solar_CMIP_SOLARIS-HEPPA-3-2_gn_185001-229912.nc",
    known_hash="4b844a3324b74be16faf7272b64ed7853ea2c606c4db841a318366ad223ebcac"
)

In [ ]:
nc = Dataset(cmip6)
tsi = nc.variables['tsi'][:]
time_cmip6 = nc.variables['time'][:]
nc.close()

In [ ]:
time_cmip6 # looks like midmonth

In [ ]:
time_cmip6 = np.arange(1850+1/24, 2300, 1/12)

In [ ]:
pl.plot(time_17495_18500, tsi_yearly_17495_18500)
pl.plot(time_cmip6, tsi)

In [ ]:
# interpolate on to month end grid
# for 1750 to 1850 linear interpolation is fine
tsi_monthend = np.zeros(12*(2300-1750))
interpolator = interp1d(time_17495_18500, tsi_yearly_17495_18500)
tsi_monthend[:100*12+1] = interpolator(np.arange(1750, 1850+1/12, 1/12))

In [ ]:
interpolator = interp1d(time_cmip6, tsi)
tsi_monthend[100*12+1:] = interpolator(np.arange(1850+1/12, 2300-0.0001, 1/12))

In [ ]:
tsi_monthend[100*12+1:]

In [ ]:
# now calculate a 1750-2014 mean
weights = np.ones(265*12+1)
weights[0] = 0.5
weights[-1] = 0.5
mean_17502014 = np.average(tsi_monthend[:(265*12)+1], weights=weights)
mean_17502014

## Calculate solar effective radiative forcing

$ \frac{1}{4} \cdot 0.71 \cdot 0.72 \left(TSI - TSI_{\mathrm{ref}} \right)$

In [ ]:
solar_erf_monthend = 0.25 * 0.71 * 0.72 * (tsi_monthend - mean_17502014)

In [ ]:
pl.plot(np.arange(1750, 2300, 1/12), solar_erf_monthend)

In [ ]:
df_out = pd.DataFrame({'solar_erf': solar_erf_monthend}, index=np.arange(1750, 2300, 1/12))
df_out

In [ ]:
os.makedirs('../data/solar-forcing')
df_out.to_csv('../data/solar-forcing/forcing_monthly_1750-2299')